# Derivatives

In this file, I will show how we can take the derivative and derivative-transpose of the `v0` tensor language.

# !!! NOTE !!! OUT OF DATE

I need to do this in terms of the updated AST / IR formulations.

In [1]:
import sys
sys.path.append('../')
sys.path.append('../src/')
#from adt import ADT
#from adt import memo as ADTmemo
from atlv0 import IR
from atlv0 import Func
from atlv0 import _Context

In [2]:
print(IR._defstr)


module IR_v0 {
    expr = Var      ( string name )
         | Const    ( float  val  )
         | Add      ( expr lhs, expr rhs )
         | Mul      ( expr lhs, expr rhs )
         | Pair     ( expr lhs, expr rhs )
         | Proj     ( int01 idx, expr arg )
         | Gen      ( string idxname, int range, expr body )
         | Sum      ( string idxname, int range, expr body )
         | Access   ( expr  base, index idx )
         -- implied multiplication of the bracket with body
         | Indicate ( pred  arg, expr body )
         -- important to express sharing of computation results
         | Let      ( string name, expr rhs, expr body )
    
    -- indices are drawn from a range s.t.
    -- 0 <= i < range
    
    pred    = Eq( index lhs, index rhs )
    
    type    = TNum    ()
            | TError  ()
            | TPair   (  type lhs, type rhs )
            | TTensor ( int range, type typ )
}



Derivatives are defined on `expr`s with respect to some number of free variables, which constitute the "inputs" to the function being differentiated.  This is expressed via the form

$$ D[[ e | X ]] $$

for which we need simply provide structurally recursive rules constituting a definition.

$$
\begin{array}{rcl}
D[[ c | X ]] &\leadsto& 0 \\
D[[ x |\ [x\mapsto dx]\in X ]] &\leadsto& dx \\
D[[ x | x\not\in X ]] &\leadsto& 0 \\
D[[ e_0 + e_1 | X ]] &\leadsto& D[[ e_0 | X ]] + D[[ e_1 | X ]] \\
D[[ e_0 \cdot e_1 | X ]] &\leadsto& D[[ e_0 | X ]] \cdot e_1 + e_0 \cdot D[[ e_1 | X ]] \\
D[[ \sum_i e | X ]] &\leadsto& \sum_i D[[ e | X ]] \\
D[[ \boxplus_i e | X ]] &\leadsto& \boxplus_i D[[ e | X ]] \\
D[[ e[i]\ | X ]] &\leadsto& (D[[ e | X ]])[i] \\
D[[\ [p]\cdot e\ | X ]] &\leadsto& [p]\cdot D[[ e | X ]] \\
D[[ (e_0,e_1) | X ]] &\leadsto& (D[[ e_0 | X ]], D[[ e_1 | X ]]) \\
D[[ \pi_k e | X ]] &\leadsto& \pi_k(D[[ e | X ]]) \\
D[[ \textrm{let } x = e_0 \textrm{ in } e_1 | X ]] &\leadsto&
    \left(\begin{array}{rcl}
    \textrm{let } x &=& e_0 \textrm{ in } \\
    \textrm{let } dx &=& D[[ e_0 | X ]] \textrm{ in } \\
     && D[[ e_1 | X[x \mapsto dx]\ ]]) \\
    \end{array}\right) \\
\end{array}
$$
